In [1]:
# Imports

In [10]:
import pyarrow as pa
import os

In [3]:
# Save an array to disk

In [5]:
example_array = pa.array([1,2,3,4,5,6])

In [7]:
schema = pa.schema([
    pa.field('numbers', example_array.type)
])

In [8]:
schema

numbers: int64

In [63]:
with pa.OSFile(os.path.join(os.getcwd(),"data","example_array.arrow"), 'wb') as sink:
    with pa.ipc.new_file(sink, schema=schema) as writer:
        batch = pa.record_batch([example_array], schema=schema)
        writer.write(batch)

In [12]:
# This is nice.  But, what abt a table?

In [13]:
a_table = pa.table([[1,2,3,4],[10,20,30,40],[100,200,300,400]],  names=["units", "tens", "huns"])

In [62]:
with pa.OSFile(os.path.join(os.getcwd(),"data","example_table_two.arrow"), 'wb') as sink:
    with pa.ipc.new_file(sink, schema = a_table.schema) as writer:
        writer.write(a_table)

In [54]:
# Alternative:  Convert to a record batch (Which requires you to extract the schema)

In [61]:
a_rb = pa.record_batch(a_table.to_pandas(), schema=a_table.schema)

with pa.OSFile(os.path.join(os.getcwd(),"data","example_table_three.arrow"), 'wb') as sink:
    with pa.ipc.new_file(sink, schema = a_table.schema) as writer:
        writer.write(a_rb)

In [60]:
## Map to Memory

In [64]:
with pa.memory_map(os.path.join(os.getcwd(),"data","example_array.arrow"), 'r') as source:
    loaded_arrays = pa.ipc.open_file(source).read_all()

In [66]:
loaded_arrays

pyarrow.Table
numbers: int64
----
numbers: [[1,2,3,4,5,6]]

In [67]:
with pa.memory_map(os.path.join(os.getcwd(),"data","example_table_two.arrow"), 'r') as source:
    loaded_table = pa.ipc.open_file(source).read_all()

In [68]:
loaded_table

pyarrow.Table
units: int64
tens: int64
huns: int64
----
units: [[1,2,3,4]]
tens: [[10,20,30,40]]
huns: [[100,200,300,400]]

In [69]:
with pa.memory_map(os.path.join(os.getcwd(),"data","example_table_three.arrow"), 'r') as source:
    loaded_table_three = pa.ipc.open_file(source).read_all()

In [70]:
loaded_table_three

pyarrow.Table
units: int64
tens: int64
huns: int64
----
units: [[1,2,3,4]]
tens: [[10,20,30,40]]
huns: [[100,200,300,400]]